In [1]:
from crm import Network

In [2]:
import re

# Data

In [3]:
f = open("data/raw/crm_output.pl")

In [4]:
edges_raw = []

In [5]:
while True:
    s = f.readline()
    if not s:
        break
    if "connected" in s:
        edges_raw.append(s)

In [6]:
edges_raw[24].split(",")[0].split("(")[-1].split(")")[0]

'1258'

In [7]:
gg = ' '.join(edges_raw[24].split(",")[1:3])

In [8]:
gg

'[a(2) a(26)]'

In [9]:
int(re.findall(r"a\([0-9]*\)", gg)[0].split("(")[-1].split(")")[0])

2

In [10]:
int(re.findall(r"a\([0-9]*\)", gg)[1].split("(")[-1].split(")")[0])

26

In [11]:
edges = []

In [12]:
for i in range(len(edges_raw)):
    end = int(edges_raw[i].split(",")[0].split("(")[-1].split(")")[0])-1
    
    gg = ' '.join(edges_raw[i].split(",")[1:3])
    start_one = int(re.findall(r"a\([0-9]*\)", gg)[0].split("(")[-1].split(")")[0])-1
    start_two = int(re.findall(r"a\([0-9]*\)", gg)[1].split("(")[-1].split(")")[0])-1
    # start_one --> end
    # start_two --> end
    edges.append((start_one, end))
    edges.append((start_two, end))

In [13]:
# edges

In [14]:
with open("data/edges.txt", "w") as fp:
    for u, v in edges:
        fp.write(f"{u} {v}\n")

In [15]:
len(edges), len(set(edges))

(2564, 2541)

In [16]:
len(edges), len(set(edges))

(2564, 2541)

In [17]:
f_mapper_global = []

In [18]:
f = open("data/raw/train_features_pos")

In [19]:
while True:
    gg = f.readline().split(" ")[3:-1]
    if not gg:
        break
    all_pos = [int(e) for e in gg]
    dd = {i: 1 if i in all_pos else 0 for i in range(1282)}
    f_mapper_global.append(dd)

In [20]:
f_mapper_global[-1]

{0: 0,
 1: 0,
 2: 0,
 3: 1,
 4: 1,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 0,
 10: 1,
 11: 0,
 12: 1,
 13: 0,
 14: 0,
 15: 1,
 16: 1,
 17: 0,
 18: 0,
 19: 0,
 20: 1,
 21: 1,
 22: 1,
 23: 1,
 24: 1,
 25: 1,
 26: 1,
 27: 1,
 28: 1,
 29: 1,
 30: 1,
 31: 1,
 32: 1,
 33: 1,
 34: 1,
 35: 0,
 36: 0,
 37: 0,
 38: 0,
 39: 0,
 40: 0,
 41: 0,
 42: 0,
 43: 0,
 44: 0,
 45: 0,
 46: 0,
 47: 0,
 48: 0,
 49: 0,
 50: 0,
 51: 0,
 52: 0,
 53: 0,
 54: 0,
 55: 0,
 56: 0,
 57: 0,
 58: 0,
 59: 0,
 60: 1,
 61: 1,
 62: 1,
 63: 1,
 64: 1,
 65: 1,
 66: 1,
 67: 0,
 68: 0,
 69: 0,
 70: 0,
 71: 1,
 72: 1,
 73: 0,
 74: 1,
 75: 1,
 76: 1,
 77: 1,
 78: 1,
 79: 0,
 80: 0,
 81: 0,
 82: 0,
 83: 0,
 84: 0,
 85: 1,
 86: 0,
 87: 0,
 88: 0,
 89: 0,
 90: 0,
 91: 0,
 92: 0,
 93: 0,
 94: 0,
 95: 0,
 96: 0,
 97: 0,
 98: 0,
 99: 0,
 100: 0,
 101: 0,
 102: 0,
 103: 0,
 104: 0,
 105: 0,
 106: 0,
 107: 0,
 108: 0,
 109: 0,
 110: 0,
 111: 0,
 112: 0,
 113: 0,
 114: 1,
 115: 1,
 116: 1,
 117: 1,
 118: 1,
 119: 1,
 120: 1,
 121: 1,
 122: 0,
 12

In [21]:
num_neurons = max([max(u, v) for u, v in edges])+1

In [22]:
adj_list = [[] for i in range(num_neurons)]

In [23]:
num_neurons

1282

In [24]:
len(adj_list)

1282

In [25]:
for u, v in edges:
    adj_list[u].append(v)

In [26]:
n = Network(num_neurons, adj_list)

In [27]:
orig_output_neurons = n.output_neurons

In [28]:
adj_list.append([])
adj_list.append([])

In [29]:
num_neurons = len(adj_list)

In [30]:
num_neurons

1284

In [31]:
for i in range(num_neurons):
    if i in orig_output_neurons:
        adj_list[i].append(num_neurons-2)
        adj_list[i].append(num_neurons-1)

In [32]:
adj_list[213]

[1282, 1283]

In [33]:
n = Network(num_neurons, adj_list)

In [34]:
n.set_neuron_activation(1282, lambda x : x, lambda x : 1)

In [35]:
n.set_neuron_activation(1283, lambda x : x, lambda x : 1)

In [54]:
# https://math.stackexchange.com/questions/945871/derivative-of-softmax-loss-function
import torch
for f in f_mapper_global:
    f[1282]=1
    f[1283]=1
    n.reset()
    o = n.forward(f)
    if o[0]!=torch.tensor([0.]):
        print(o)
        break

In [60]:
m = torch.nn.Sequential(torch.nn.Linear(10, 100))

In [72]:
list(m.parameters())[1]

Parameter containing:
tensor([ 0.2583, -0.2650, -0.1380,  0.2627, -0.0312,  0.1305, -0.2825,  0.2821,
        -0.1505, -0.1986,  0.1177, -0.0656, -0.2120,  0.2727,  0.2055,  0.2070,
         0.0710,  0.1732,  0.0441,  0.2178,  0.0877,  0.0890, -0.2641,  0.1323,
         0.2862,  0.1050, -0.0550,  0.0561,  0.1339,  0.1444, -0.1224,  0.0247,
        -0.2285, -0.1712,  0.2991,  0.2760,  0.3068, -0.2236,  0.1844,  0.2605,
         0.1401, -0.1417,  0.0496, -0.2446,  0.2426,  0.2560,  0.0176, -0.0214,
         0.0734, -0.2629, -0.1454, -0.0164, -0.2263, -0.1252,  0.2839,  0.1112,
        -0.1526, -0.2420,  0.1288, -0.2328, -0.1631, -0.2980, -0.2495,  0.0282,
         0.2589,  0.1794, -0.2060,  0.0578, -0.1831, -0.0485,  0.1794,  0.1493,
        -0.2882, -0.2283,  0.1230, -0.0346,  0.0167, -0.0178,  0.0618,  0.3064,
         0.2408, -0.1345,  0.1730,  0.0623,  0.2448,  0.2489,  0.2486, -0.2769,
         0.0622, -0.0840, -0.0709,  0.1767, -0.0693,  0.1541,  0.2729,  0.1055,
         0.1617, -

In [48]:
import torch

In [74]:
n = Network(4, [[1], [2], [3], []])
n.weights = {
        (0, 1): torch.tensor(1., requires_grad=True),
        (1, 2): torch.tensor(2., requires_grad=True),
        (2, 3): torch.tensor(3., requires_grad=True),
}

In [75]:
n.weights[(2, 3)], n.weights[(2, 3)].grad

(tensor(3., requires_grad=True), None)

In [76]:
n_o = n.forward({0:10, 1:1, 2:1, 3:1})

In [77]:
n_o

tensor([60.], grad_fn=<StackBackward>)

In [78]:
loss = 100 - n_o

In [79]:
loss

tensor([40.], grad_fn=<RsubBackward1>)

In [80]:
loss.backward()

In [88]:
n.weights[(2, 3)].grad

tensor(-20.)

In [89]:
optimizer = torch.optim.Adam((v for v in n.weights.values()), lr=0.1)

In [90]:
optimizer.zero_grad()

In [91]:
n.weights[(2, 3)].grad

tensor(0.)

In [81]:
n.weights[(2, 3)], n.weights[(2, 3)].grad

(tensor(3., requires_grad=True), tensor(-20.))

In [95]:
loss.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [92]:
a.grad

tensor(30.)

In [87]:
torch.stack([o1, o2]).backward()

RuntimeError: grad can be implicitly created only for scalar outputs